In [12]:
print("Hello")

Hello


# Connect to Summit MAIN GCDS via REST and read trade and its counterparty

In [13]:
import requests
import json

with open("secrets.json") as file:
    credentials = json.load(file)
api_url = credentials['summit_rest_url']
print("Summit REST URL:", api_url)

# Do the Summit connection
get_trade_url = api_url + "/entity/SWAP?TradeId=23548S&TradeType=SWAP"
print("REST endpoint:", get_trade_url)

payload = ""
headers = {}
response = requests.request("GET", get_trade_url, headers=headers, data=payload)
print("Response from Summit:", response.text)

# Get Counterparty from Envelope of trade
trade_info = json.loads(response.text)
print(trade_info)
counterparty = trade_info["Env"]["Cust"]
print("Counterparty retrieved:", counterparty)

# Get Full legal name from Counterparty record
get_cp_url = api_url + "/entity/CUSTOMER?Id=" + counterparty
print("REST endpoint:", get_cp_url)

response = requests.request("GET", get_cp_url, headers=headers, data=payload)
print("Response from Summit:", response.text)
cp_info = json.loads(response.text)
print(cp_info)
cp_legal_name = cp_info["LegalName"]
print("Legal name retrieved:", cp_legal_name)


Summit REST URL: https://summit-main.cdsdemo.dhltd.com
REST endpoint: https://summit-main.cdsdemo.dhltd.com/entity/SWAP?TradeId=23548S&TradeType=SWAP
Response from Summit: {"TradeId":"23548S","TradeType":"SWAP","Env":{"Audit_Id":"000009YH","Audit_Table":"SWAP","Audit_Version":14,"Audit_Current":"Y","Audit_EntityState":"VER","Audit_Authorized":"Y","Audit_PendingAction":"","Audit_PendingState":"","Audit_IsHistory":"N","Audit_User":"stp_user","Audit_Action":"ACCEPT","Audit_DateTime":"2023-07-06T13:13:04","Audit_ExtStateName":"Verified","Audit_StateId":0,"dmOwnerTable":"SWAP","TradeId":"23548S","InputDate":"2020-04-09","InputTime":"13:39:47","Cust":"BNYM","Company":"SUMMIT","BookedBy":"","BookedFor":"","BookSpr":0,"BookCcy":"","BookFee":0,"BookFeeDate":"","RC":"","Book":"IRD_FLOW","RCSpread":0,"Trader":"","Broker":"","Contact":"","Desk":"MARKETS","Folder":"","DealId":"","TradeDate":"2020-04-09","TradeStatus":"VER","RCTrade":"N","ProductGroup":"","ProductType":"","BackDated":"Y","FinalMatur

# Use the counterparty legal name and ask GPT-4 to give info in 30 words about the credit risk of counterparty

In [17]:
import openai
import json

with open("secrets.json") as file:
    credentials = json.load(file)
openai.api_key = credentials['openai_api_key']

model = "davinci"
question = "What is the current Moody's and S&P credit rating of " + cp_legal_name
print("Question:", question)

response = openai.Completion.create(
    engine=model,
    prompt=(f"Question: {question}\n"
            "Answer:"
            ),
    max_tokens=100,
    n=1,
    stop=None,
    temperature=0.5
)

answer = response.choices[0].text.split('\n')[0]
print("Answer with model", model,":", answer)


Question: What is the current Moody's and S&P credit rating of Bank of New York Mellon Corporation
Answer with model davinci :  Moody's and S&P both rate BNY Mellon as A1, which is an excellent credit rating.


# Use LangChain to google counterparty and have GPT-4 check if results are credit risk related